In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/itv020649/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
spark

In [3]:
orderDf = spark.read \
.format("csv") \
.option("inferSchema","true") \
.option("header","true") \
.load("/public/trendytech/orders_wh/*")

In [5]:
orderDf.show(10)

+--------+--------------------+-----------+---------------+
|order_id|          order_date|customer_id|   order_status|
+--------+--------------------+-----------+---------------+
|       1|2013-07-25 00:00:...|      11599|         CLOSED|
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|      12111|       COMPLETE|
|       4|2013-07-25 00:00:...|       8827|         CLOSED|
|       5|2013-07-25 00:00:...|      11318|       COMPLETE|
|       6|2013-07-25 00:00:...|       7130|       COMPLETE|
|       7|2013-07-25 00:00:...|       4530|       COMPLETE|
|       8|2013-07-25 00:00:...|       2911|     PROCESSING|
|       9|2013-07-25 00:00:...|       5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|       5648|PENDING_PAYMENT|
+--------+--------------------+-----------+---------------+
only showing top 10 rows



In [6]:
orderDf.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)



## Optimization session 1
1. application code level
cache, reducebykey > gropupby key, parquet
2. cluster level optimization
resource disturbution, containers/executers

### session 2
resources - memory (RAM) , CPU cores (Compute)
our intention to make sure job is getting right amount of resources

10 node cluster (10 worker node)
16 CPU core
64 GB RAM

Executor (container of resource)
1 node > 1 executor(multiple container in one node)

container/executor/JVM - cpu core  = memory(ram)

16 core, 64 GB RAM - 1 core for backgroud processes, 15 core usable

1. Thin executor - to create more executor with minimum possible executor.
16 executor, with each holding 1 core and 4 GB RAM
Drawback:-
multi-threading not possible
a lot of copy of broadcast variable required


2. Fat executor - As big as possible
1 executor : 16 core, 64 GB ram
Draback:
it is observed, if ececutor hold more than 5 core HDFS throughput suffers.
if execotor holds very huge amount of memory, garbage collection take a lot time.
garbage collection:removing objects from memory.



### Session 3

Right aproach balanced approach
16 cores,64GB
1gb OS
1 core to demon process

Cores >1 support multithreading, HDFS trouughput should not suffer cores<=5
3 executor: each 5 cpu core and 21 GB
out of 21 GB some of it will go to OFF heap(overhead)
MAX(384, 7% of executor memory) = 1.5GB
21-1.5 ~ 19 GB

5 CPU core, 19 GB RAM
10 node clustor/worker node
10*3=30 30 executor

1 Executor will be taken by YARN application manager
10 nodes * 3 executor * 5 CPU Core = 150 tasks will run in parallel

on-heap - memory inside executor
Off-heap memory - outside executor   (objects created hear doesnot require Garbage collection but memory should be manageg programatically)

